04_11_EXOtic_preparation

### 모듈 버전 확인

아래 셀을 실행하면 이 노트북을 실행한 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

https://github.com/rzellem/EXOTIC

In [68]:
!pip install photutils==1.12

In [69]:
import importlib, sys, subprocess
packages = "ysfitsutilpy, ysphotutilpy, version_information" # required modules
pkgs = packages.split(", ")
for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        print(f"**** module {pkg} is being installed")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
    else: 
        print(f"**** module {pkg} is installed")

%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

**** module ysfitsutilpy is installed
**** module ysphotutilpy is installed
**** module version_information is installed
The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information
This notebook was generated at 2025-01-20 02:33:54 (KST = GMT+0900) 
0 Python     3.10.16 64bit [GCC 11.2.0]
1 IPython    8.27.0
2 OS         Linux 6.8.0 51 generic x86_64 with glibc2.39
3 ysfitsutilpy 0.2
4 ysphotutilpy 0.1.1
5 version_information 1.0.4


### import modules

In [70]:
from glob import glob
from pathlib import Path
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas  as pd
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import EarthLocation
from astroquery.simbad import Simbad

import json

import ysfitsutilpy as yfu

import _astro_utilities
import _Python_utilities

plt.rcParams.update({'figure.max_open_warning': 0})

import warnings
warnings.filterwarnings('ignore')

## 변수 설정

In [71]:
count_stars = False
verbose = True
tryagain = False
trynightsky = False
tryASTROMETRYNET = True
# file_age = 365
# downsample = 4

## 프로젝트 폴더 

그리고 프로젝트 마다 폴더를 구분하여 저장하고 있으며, 프로젝트 폴더마다 전처리를 동일한 파일로 수행하는 방법을 택하고 있습니다. 
전처리 파일은 "CAL-BDF" 폴더에 모두 넣어두고 마스터 파일을 만들어서 사용합니다. 이때 폴더명/파일명은 pathlib을 이용하여 처리합니다.

In [72]:
#%%
#######################################################
BASEDIR = Path("/mnt/Rdata/ASTRO_data")  

PROJECDIR = BASEDIR / "C3-EXO"
TODODIR = PROJECDIR / "-_-_-_2024-05_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-05_-_RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-09_-_RiLA600_ASI6200MMPro_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2024-11_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-11_-_RiLA600_ASI6200MMPro_-_3bin"
# TODODIR = PROJECDIR / "-_-_-_2025-01_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2025-01_-_RiLA600_ASI6200MMPro_-_3bin"

# PROJECDIR = BASEDIR / "C4-Spectra"
# TODODIR = PROJECDIR / "-_-_-_2024-05_TEC140_ASI183MMPro_-_1bin"

# PROJECDIR = BASEDIR / "C5-Test"
# TODODIR = PROJECDIR / "-_-_-_-_GSON300_STF-8300M_-_1bin"

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(TODODIR))
if verbose == True :
    print ("DOINGDIRs: ", format(DOINGDIRs))
    print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

try : 
    BDFDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
    print ("BDFDIR: ", format(BDFDIR))
    BDFDIR = Path(BDFDIR[0])    
except : 
    BDFDIR = TODODIR
    pass

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
# print ("DOINGDIRs: ", format(DOINGDIRs))
# print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

# filter_str = 'TrES-1b'
# DOINGDIRs = [x for x in DOINGDIRs if filter_str in str(x)]
# remove = 'BIAS'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'DARK'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'FLAT'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
if verbose == True :
    print ("DOINGDIRs: ", DOINGDIRs)
    print ("len(DOINGDIRs): ", len(DOINGDIRs))
#######################################################

DOINGDIRs:  ['/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin/-_CAL-BDF_-_2024-05_-_-_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin/HAT-P-37b_LIGHT_-_2024-06-28_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin/HD189733b_LIGHT_-_2024-06-27_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin/Kepler-17b_LIGHT_-_2024-06-26_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin/TrES-1b_LIGHT_-_2024-06-09_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin/TrES-1b_LIGHT_-_2024-06-18_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin/TrES-4b_LIGHT_-_2024-06-13_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin/WASP-103b_LIGHT_-_2024-06-

## 환경 변수 설정

In [73]:
#####################################################################
# Observed location
LOCATION = dict(lon=127.005, lat=37.308889, elevation=101)
Suwon =  EarthLocation(lon=127.005 * u.deg, 
                                 lat=37.308889 * u.deg, 
                                 height=101 * u.m)
observatory_code = "P64"

# Used for any `astropy.SkyCoord` object:
SKYC_KW = dict(unit=u.deg, frame='icrs')
#######################################################
# Initial guess of FWHM in pixel
FWHM_INIT = 4

Mag_Low = 11.5
Mag_High = 15

Mag_target = 12.5
Mag_delta = 2
ERR_Max = 0.5

coord_deltas = np.arange(0.00001, 0.00050, 0.00001)
#######################################################

In [74]:
for DOINGDIR in DOINGDIRs :
    # DOINGDIR = DOINGDIRs[0]
    DOINGDIR = Path(DOINGDIR)
    foldername_el = DOINGDIR.parts[-1].split("_")
    targ_name = foldername_el[0]
    # targ_name = targ_name.replace("-"," ")
    # targ_name = ''.join([i for i in targ_name  if not i.isdigit()])
    print("targ_name :", targ_name)

    obsdate = foldername_el[3]
    bin = foldername_el[-1][0]

    result_table = Simbad.query_object(targ_name)
    if verbose ==  True :
        print(f"result_table :\n{result_table}")
        print(f"RA: {result_table['RA'][0]}, DEC: {result_table['DEC'][0]}")

    targ_sky = SkyCoord(ra=result_table['RA'][0],
                        dec=result_table['DEC'][0], 
                        unit=(u.hourangle, u.degree),
                        frame='icrs')
    if verbose ==  True :
        print("targ_sky :", targ_sky)

    df = pd.read_csv("PSCompPars.csv",
                    skiprows=322)

    coord_delta = 0.0001
    df_targ = df.loc[(df["ra"] > targ_sky.ra.value*(1-coord_delta)) \
                    & (df["ra"] < targ_sky.ra.value*(1+coord_delta)) \
                    & (df["dec"] > targ_sky.dec.value*(1-coord_delta))\
                    & (df["dec"] < targ_sky.dec.value*(1+coord_delta))]
    df_targ = df_targ.reset_index()
    if verbose ==  True :
        print("df_targ['pl_name']: {df_targ['pl_name']}")


    if verbose == True :
        print("DOINGDIR :", DOINGDIR)
    REDUCEDIR = DOINGDIR / _astro_utilities.reduced_dir

    summary = yfu.make_summary(REDUCEDIR/"*.fit*", 
                                    verify_fix=True,
                                    ignore_missing_simple=True,
                            )
    if summary is not None :
        if verbose == True :
            #print(summary)
            print("len(summary):", len(summary))
            # print("summary:", summary)
            #print(summary["file"][0])

        check_filters = summary['FILTER'].drop_duplicates()
        check_filters = check_filters.reset_index(drop=True)
        if verbose == True :
            print(f"check_filters :\n{check_filters}")

        for filt in check_filters:
            print(filt)
            summary_filt = summary.loc[summary["FILTER"] == filt].copy()
            print(summary_filt)
            print("type(summary_filt)) :", type(summary_filt))
            # summary_filt = summary_filt.reset_index(inplace=True)
            # print(summary_filt)
            
            if verbose == True :
                print("type(summary_filt)) :", type(summary_filt))

            if summary_filt is not None :

                FILTDIR = REDUCEDIR.parent / f"EXOTIC_{filt}"
                EXOFINAL = REDUCEDIR.parent / f"EXOTIC_{filt}_result"

                if verbose == True :
                    print(FILTDIR)
                    print(EXOFINAL)
                if not FILTDIR.exists() : 
                    os.mkdir(str(FILTDIR))
                    if verbose == True :
                        print(f"{FILTDIR} is created")
                if not EXOFINAL.exists() : 
                    os.mkdir(str(EXOFINAL))
                    if verbose == True :
                        print(f"{EXOFINAL} is created")


                # From a string:
                json_string = '''
                {
                    "user_info": {
                            "Directory with FITS files": " ",
                            "Directory to Save Plots": " ",
                            "Directory of Flats": null,
                            "Directory of Darks": null,
                            "Directory of Biases": null,

                            "AAVSO Observer Code (N/A if none)": "PKIE",
                            "Secondary Observer Codes (N/A if none)": "N/A",

                            "Observation date": "",
                            "Obs. Latitude": "+37.309",
                            "Obs. Longitude": "+127.005",
                            "Obs. Elevation (meters)": "101",
                            "Camera Type (CCD or DSLR)": "CCD",
                            "Pixel Binning": "1x1",
                            "Filter Name (aavso.org/filters)": "V",
                            "Observing Notes": "Weather, seeing was nice.",

                            "Plate Solution? (y/n)": "n",

                            "Target Star X & Y Pixel": null,
                            "Comparison Star(s) X & Y Pixel": null
                    },
                    "planetary_parameters": {
                            "Target Star RA": "02:04:10",
                            "Target Star Dec": "+46:41:23",
                            "Planet Name": "HAT-P-32 b",
                            "Host Star Name": "HAT-P-32",
                            "Orbital Period (days)": 2.1500082,
                            "Orbital Period Uncertainty": null,
                            "Published Mid-Transit Time (BJD-UTC)": null,
                            "Mid-Transit Time Uncertainty": null,
                            "Ratio of Planet to Stellar Radius (Rp/Rs)": null,
                            "Ratio of Planet to Stellar Radius (Rp/Rs) Uncertainty": null,
                            "Ratio of Distance to Stellar Radius (a/Rs)": null,
                            "Ratio of Distance to Stellar Radius (a/Rs) Uncertainty": null,
                            "Orbital Inclination (deg)": null,
                            "Orbital Inclination (deg) Uncertainity": null,
                            "Orbital Eccentricity (0 if null)": null,
                            "Star Effective Temperature (K)": null,
                            "Star Effective Temperature (+) Uncertainty": null,
                            "Star Effective Temperature (-) Uncertainty": null,
                            "Star Metallicity ([FE/H])": null,
                            "Star Metallicity (+) Uncertainty": null,
                            "Star Metallicity (-) Uncertainty": null,
                            "Star Surface Gravity (log(g))": null,
                            "Star Surface Gravity (+) Uncertainty": null,
                            "Star Surface Gravity (-) Uncertainty": null
                    },
                    "optional_info": {
                            "Pixel Scale (Ex: 5.21 arcsecs/pixel)": null,
                            "Filter Minimum Wavelength (nm)": null,
                            "Filter Maximum Wavelength (nm)": null
                    }
                }'''
                init_json = json.loads(json_string)

                init_json['user_info']['Directory with FITS files'] = f"{FILTDIR}"
                init_json['user_info']['Directory to Save Plots'] = f"{EXOFINAL}"
                init_json['user_info']['Observation date'] = f"{obsdate}"
                init_json['user_info']['Pixel Binning'] = f"{bin}x{bin}"
                init_json['user_info']['Filter Name (aavso.org/filters)'] = f"{filt}", 
                init_json['planetary_parameters']['Target Star RA'] = f"{df_targ['ra'][0]}"
                init_json['planetary_parameters']['Target Star Dec'] = f"{df_targ['dec'][0]}"
                init_json['planetary_parameters']['Planet Name'] = f"{df_targ['pl_name'][0]}"
                init_json['planetary_parameters']['Host Star Name'] = f"{df_targ['hostname'][0]}"
                if verbose == True : 
                    print("init_json :\n{init_json}")

                with open(f"{EXOFINAL}/init.json", "w") as f:
                    json.dump(init_json, f, indent=4) # indent
                    print("init_json is created...")
                
                for idx, row in summary_filt.iterrows() :
                    fpath = Path(row['file'])
                    if not (FILTDIR/fpath.name).exists() :
                        shutil.copy2(fpath, FILTDIR/fpath.name) 
                        if verbose == True :
                            print(f"{fpath} is copy to {FILTDIR/fpath.name}")
                    


targ_name : HAT-P-37b
result_table :
 MAIN_ID        RA           DEC      ...     COO_BIBCODE     SCRIPT_NUMBER_ID
             "h:m:s"       "d:m:s"    ...                                     
--------- ------------- ------------- ... ------------------- ----------------
HAT-P-37b 18 57 11.0562 +51 16 08.850 ... 2020yCat.1350....0G                1
RA: 18 57 11.0562, DEC: +51 16 08.850
targ_sky : <SkyCoord (ICRS): (ra, dec) in deg
    (284.2960675, 51.269125)>
df_targ['pl_name']: {df_targ['pl_name']}
DOINGDIR : /mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin/HAT-P-37b_LIGHT_-_2024-06-28_-_RiLA600_STX-16803_-_2bin
All 131 keywords (guessed from /mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin/HAT-P-37b_LIGHT_-_2024-06-28_-_RiLA600_STX-16803_-_2bin/reduced/HAT-P-37b_LIGHT_R_2024-06-28-12-19-58_210sec_RiLA600_STX-16803_-17c_2bin.fit) will be loaded.
len(summary): 96
check_filters :
0    R
1    V
Name: FILTER, dtype: object
R
                   